In [1]:
import string
import os
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import re
from nltk.tokenize import TweetTokenizer

SOS = '<'
EOS = '>'

def preprocess_line(line):
    """Препроцессинг строки текста."""
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(line.lower())
    return " ".join(tokens)

def get_poems_list(text):
    """Преобразует текст стихов в список."""
    poems = [""]
    current_poem_index = 0
    new_poem = True

    for line in text:
        # Пропускаем короткие строки и пустые строки
        if len(line) < 10 or line == '\n':
            if not new_poem:
                # Завершаем текущее стихотворение
                poem_list = list(poems[current_poem_index])
                poem_list[-1] = EOS
                poems[current_poem_index] = "".join(poem_list)
                poems.append("")
                current_poem_index += 1
                new_poem = True
            continue

        # Начинаем новое стихотворение или строку в текущем стихотворении
        if new_poem:
            poems[current_poem_index] += SOS + " "
        else:
            poems[current_poem_index] += "* "

        new_poem = False

        # Препроцессинг и добавление строки к текущему стихотворению
        line = line.strip()
        line = re.sub(r'\s+', ' ', line)
        line += "\n"
        poems[current_poem_index] += preprocess_line(line)

    return poems


In [ ]:
with open('/content/Esenin.txt', 'r',encoding='utf-8') as iofile:
    text_onegin = iofile.readlines()

text_esenin = [x.replace('\t\t', '') for x in text_onegin]
text_esenin_new = " ".join(text_onegin)
print(text_esenin_new[0:300])

In [ ]:
cd

/root


In [ ]:
poems = get_poems_list(text_esenin)

In [ ]:
all_string = ""
for line in text_onegin:
  all_string += line

text_all = all_string.lower()

In [ ]:
text_all

'поэт\n\n\t\tон бледен. мыслит страшный путь.\n\t\tв его душе живут виденья.\n\t\tударом жизни вбита грудь,\n\t\tа щеки выпили сомненья.\n\n\t\tклоками сбиты волоса,\n\t\tчело высокое в морщинах,\n\t\tно ясных грез его краса\n\t\tгорит в продуманных картинах.\n\n\t\tсидит он в тесном чердаке,\n\t\tогарок свечки режет взоры,\n\t\tа карандаш в его руке\n\t\tведет с ним тайно разговоры.\n\n\t\tон пишет песню грустных дум,\n\t\tон ловит сердцем тень былого.\n\t\tи этот шум… душевный шум…\n\t\tснесет он завтра за целковый.\n\n\t\t1910—1912\n\n\nночь\n\n\t\tусталый день склонился к ночи,\n\t\tзатихла шумная волна,\n\t\tпогасло солнце, и над миром\n\t\tплывет задумчиво луна.\n\t\tдолина тихая внимает\n\t\tжурчанью мирного ручья.\n\t\tи темный лес, склоняся, дремлет\n\t\tпод звуки песен соловья.\n\t\tвнимая песням, с берегами,\n\t\tласкаясь, шепчется река.\n\t\tи тихо слышится над нею\n\t\tвеселый шелест тростника.\n\n\t\t1911—1912\n\n\nзвезды\n\n\t\tзвездочки ясные, звезды высокие!\n\t\tчто в

In [ ]:
PAD = '_'
BEGIN_LINE = '*'
out = text_all
tokens = sorted(set(out))
ADD_TOKENS = [SOS, EOS, BEGIN_LINE, PAD]
tokens.extend([SOS, EOS, BEGIN_LINE, PAD])
print(tokens)
num_tokens = len(tokens)
print(num_tokens)


['\t', '\n', ' ', '!', '#', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', '[', ']', '_', 'c', 'i', 'm', 'p', 's', 'v', 'x', '\xa0', '«', '»', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '–', '—', '“', '„', '…', '<', '>', '*', '_']
83


In [ ]:
# dict <index>:<char>
token_to_id = {token: idx for idx, token in enumerate(tokens)}

# dict <char>:<index>
idx_to_token = {idx: token for idx, token in enumerate(tokens)}

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F

In [ ]:
def to_matrix(sentences, max_len = None, pad=PAD, dtype='int32', batch_first=True):

    max_len = max_len or max(map(len, sentences))
    sentences_ix = np.zeros([len(sentences), max_len], dtype) + token_to_id[PAD]

    for i in range(len(sentences)):
        line_ix = [token_to_id[c] for c in sentences[i]]
        sentences_ix[i, :len(line_ix)] = line_ix[:max_len]
    if not batch_first:
        sentences_ix = np.transpose(sentences_ix)
    return sentences_ix

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from mlpractice.rnn_torch import CharRNNCell
class CharRNNCell(nn.Module):

    def __init__(self, num_tokens=len(tokens), embedding_size=100, rnn_num_units=300):
        super(self.__class__, self).__init__()

        # Определение числа нейронов в RNN
        self.num_units = rnn_num_units

        # Создание слоя для эмбеддингов
        self.embedding = nn.Embedding(num_tokens, embedding_size)

        # Создание слоя для обновления состояния RNN
        self.rnn_update = nn.Linear(
            embedding_size + rnn_num_units,
            rnn_num_units
        )

        # Создание слоя для преобразования состояния RNN в логиты (выходные значения)
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)

    def forward(self, x, h_prev):
        """
        Этот метод вычисляет h_next(x, h_prev) и log P(x_next | h_next).
        Мы будем вызывать его многократно, чтобы получить всю последовательность.

        :param x: пакет идентификаторов символов, содержащий вектор int64
        :param h_prev: предыдущие скрытые состояния RNN, содержащие матрицу [batch, rnn_num_units] с плавающей запятой
        """

        # Получение эмбеддингов для входных символов
        x_emb = self.embedding(x)

        # Объединение эмбеддингов и предыдущего скрытого состояния RNN
        x_and_h = torch.cat([x_emb, h_prev], dim=-1)

        # Обновление состояния RNN
        h_next = self.rnn_update(x_and_h)
        h_next = torch.tanh(h_next)

        # Проверка размерности нового состояния
        assert h_next.size() == h_prev.size()

        # Преобразование состояния RNN в логиты
        logits = self.rnn_to_logits(h_next)

        # Применение softmax для получения вероятностного распределения
        return h_next, F.log_softmax(logits, -1)

    def initial_state(self, batch_size):
        """Возвращает начальное состояние RNN перед обработкой первого входа (h0)"""
        return torch.zeros(batch_size, self.num_units, requires_grad=True)


In [ ]:
char_rnn = CharRNNCell()
#функция правдоподобия
criterion = nn.NLLLoss()

In [ ]:
def rnn_loop(char_rnn, batch_ix):
    """
    Вычисляет log P(next_character) для всех временных шагов в names_ix
    :param batch_ix: матрица int32 формы [batch, time], результат функции to_matrix(names)
    """
    # Получение размеров бача и максимальной длины
    batch_size, max_length = batch_ix.size()

    # Инициализация скрытого состояния RNN
    hid_state = char_rnn.initial_state(batch_size)

    # Список для сохранения логарифмов вероятностей на каждом временном шаге
    logprobs = []

    # Итерация по временным шагам (по столбцам матрицы)
    for x_t in batch_ix.transpose(0, 1):
        # Обновление скрытого состояния и получение логарифмов вероятностей для следующего символа
        hid_state, logp_next = char_rnn(x_t, hid_state)

        # Добавление логарифмов вероятностей в список
        logprobs.append(logp_next)

    # Преобразование списка в тензор
    return torch.stack(logprobs, dim=1)


In [ ]:
from IPython.display import clear_output
from random import sample

char_rnn = CharRNNCell()
criterion = nn.NLLLoss()
history = []

In [ ]:
poems = poems.copy()

In [ ]:
mkdir ='/content/drive/MyDrive/Poems'
path = '/content/drive/MyDrive/Poems'

In [ ]:
import pickle

opt = torch.optim.Adam(char_rnn.parameters(), lr=1e-4, amsgrad=True)
#opt = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, 100, 1)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#    opt,
#    patience=10,
#    verbose=True,
#    factor=0.5,
#    threshold = 0.0001
#)

for i in range(5000):
    batch_ix = to_matrix(sample(poems, 32))
    batch_ix = torch.tensor(batch_ix, dtype=torch.int64)
    logp_seq = rnn_loop(char_rnn, batch_ix)
    # compute loss
    predictions_logp = logp_seq[:, :-1]
    actual_next_tokens = batch_ix[:, 1:]
    loss = criterion(predictions_logp.contiguous().view(-1, num_tokens),
                  actual_next_tokens.contiguous().view(-1))
    # train with backprop
    loss.backward()
    opt.step()
    opt.zero_grad()
    loss_value = loss.data.numpy()
    scheduler.step(loss_value)
    history.append(loss.data.numpy())
    if (i+1)%100==0:
        clear_output(True)
        plt.plot(history,label='loss')
        plt.legend()
        plt.show()

    if (i+1)%500==0:
        with open(f"{path}/char_rnn_onegin{loss_value}.pth", "wb") as fp:
                 torch.save(char_rnn.state_dict(), fp)

assert np.mean(history[:10]) > np.mean(history[-10:]), "RNN didn't converge."

In [ ]:
model = CharRNNCell()
with open(f"/content/drive/MyDrive/Poems/char_rnn_onegin0.3910471498966217.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)
MAX_LENGTH = 300

In [ ]:
def generate_sample(char_rnn, seed_phrase='  ', max_length= MAX_LENGTH, temperature=1.0):

    x_sequence = [token_to_id[token] for token in seed_phrase]
    x_sequence = torch.tensor([x_sequence], dtype=torch.int64)
    hid_state = char_rnn.initial_state(batch_size=1)

    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        hid_state, out = char_rnn(x_sequence[:, i], hid_state)

    #start generating
    for _ in range(max_length - len(seed_phrase)):
        hid_state, out = char_rnn(x_sequence[:, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.numpy()[0]

        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(len(tokens), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
    result = ''.join([tokens[ix] for ix in x_sequence.data.numpy()[0]])
    return result

Температура 0.1

In [ ]:
result = generate_sample(char_rnn, seed_phrase='< не спаивать сердце стравит страдать с половиной \n', max_length= 1000, temperature=0.1)

In [ ]:
ind = result.find('>')
print(result[0: ind + 1])

In [ ]:
print(result[0: ind + 1])

Температура 0.5

In [ ]:
result = (generate_sample(char_rnn, seed_phrase='< пускай я порою от спирта вымок \n', max_length= 1000, temperature=0.5))
ind = result.find('>')
print(result[0: ind + 1])



In [ ]:
пускай я порою от спирта вымок
* не спаивать сердце стравит страдать с половиной
* пускай не люблю жизть
* лелея весну